# Person Detection and Saving Person Crops

In [ ]:
## Install packages if not installed
#!pip install torch torchvision ultralytics

In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
import glob
from tqdm import tqdm
import math

from ultralytics import YOLO
import torch

# set device
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [ ]:
MODEL_NAME = "yolov8l.pt" ## will be automatically downloaded

model = YOLO(MODEL_NAME)
model.to(device)

## Detect persons in an image

In [ ]:
MIN_CONF = 0.7 # minimum confidence for detection
MIN_WIDTH = 80 # minimum width of person crop to save
MIN_HEIGHT = 80 # minimum height of person crop to save
SAVE_QUALITY = 90 # quality of save crops, reduce for large images to save disk space
SAVE_DIR = "output"
FILE_PATH = "assets/example1.jpg"

file_name = os.path.basename(FILE_PATH)

image_cv = cv2.imread(FILE_PATH)
results = model.predict(image_cv, conf=MIN_CONF, verbose=False)

for result in results:
    for i, cls in enumerate(result.boxes.cls):
        if int(cls) == 0:
            xywh = result.boxes.xywh[i]
            width_p =  math.floor(xywh[2])
            height_p = math.floor(xywh[3])

            if width_p > MIN_WIDTH and height_p > MIN_HEIGHT:
                xyxy = result.boxes.xyxy[i]
                conf = result.boxes.conf[i]
                x1, y1, x2, y2 = math.floor(xyxy[0]), math.floor(xyxy[1]), math.floor(xyxy[2]), math.floor(xyxy[3])
                
                person_crop_cv = image_cv[y1:y2, x1:x2]

                crop_save_name = file_name[:-4] + '_' \
                                    + str(i) + \
                                    '_conf_' + str(int(conf*100)) + \
                                    '_bbox_' + str(x1) + '_' + str(y1) + '_' +  str(x2) + '_' + str(y2) + \
                                    '_size_' + str(width_p) + 'x' + str(height_p) + '.jpg'
                cv2.imwrite(os.path.join("output", crop_save_name), person_crop_cv, [cv2.IMWRITE_JPEG_QUALITY, SAVE_QUALITY])